In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from transfer_func_models.GA_polynomial import *

In [3]:
# Representing the following polynomial as a candPolynomial().
# p(a, b, c) = 7 + 0.8*a^2 + 9*c^0.2 + 1.2*(b^2)*(c^0.3)

var_names = ["a", "b", "c"]
vars_dict = {"a" : 0.4, "b" : 0.3, "c" : 0.7}

candPoly = candPolynomial(var_names, order = "second")

def true_polynomial(vars_dict):
    # sanity check function
    return 0 + 0.8*vars_dict["a"]**2 + 2*vars_dict["c"]**0.2 + 1.2*(vars_dict["b"]**2)*(vars_dict["c"]**0.3)

params_dict = {}
params_dict["offset"] = 0
params_dict["first_order_keeps"] = np.array([1, 0, 1])
params_dict["first_order_coeffs"] = np.array([0.8, 0, 2])
params_dict["first_order_exps"] = np.array([2, 0, 0.2])
params_dict["second_order_keeps"] = np.array([0, 0, 1])
params_dict["second_order_coeffs"] = np.array([0, 0, 1.2])
params_dict["second_order_exps"] = np.array([0, 0, 0, 0, 2, 0.3])

candPoly.set_params(params_dict)

print(true_polynomial(vars_dict))
print(candPoly.compute(vars_dict))

2.0873403619030646
2.0873403619030646


In [4]:
# now consider trying to model the function
# f(x; a, b, c) = x^p(a, b, c)
# where p(a, b, c) is the same as above

# generating example data
datapoints = []

n_datapoints = 30
Xs = np.linspace(0.5, 2.5, 100)

for i in range(n_datapoints):
    a = np.random.uniform(0, 1)
    b = np.random.uniform(0, 1)
    c = np.random.uniform(0, 1)
    print(f"Generating data with a = {a}, b = {b}, c = {c} ...")
    vars_dict = {"a" : a, "b" : b, "c" : c}
    Ys = Xs**true_polynomial(vars_dict)
    #print(true_polynomial(vars_dict))
    
    datapoints.append(dataPoint(vars_dict, Xs, Ys))


Generating data with a = 0.9830318892671555, b = 0.6562965089286218, c = 0.41515343773157254 ...
Generating data with a = 0.07497145502343305, b = 0.12082379636279694, c = 0.6961853971320422 ...
Generating data with a = 0.24627298259464148, b = 0.821678802507645, c = 0.5100396948063056 ...
Generating data with a = 0.38793051198696815, b = 0.8445653091189447, c = 0.712824617745938 ...
Generating data with a = 0.5029918956443729, b = 0.3833874814588274, c = 0.7648097847150594 ...
Generating data with a = 0.5673734549576915, b = 0.8396386435694695, c = 0.6054284654528123 ...
Generating data with a = 0.23844722108256455, b = 0.16223763944311065, c = 0.439226123114589 ...
Generating data with a = 0.18610810601296202, b = 0.37067671437277994, c = 0.3778782354032608 ...
Generating data with a = 0.0980278027901097, b = 0.5029228042055005, c = 0.7263913657688656 ...
Generating data with a = 0.6858924946545895, b = 0.9645649320361327, c = 0.8545863598406579 ...
Generating data with a = 0.3224954

In [5]:
ranges_dict = {}

ranges_dict["offset_range"] = np.array([-10, 10]) 
        
ranges_dict["first_order_coeff_ranges"] = np.array([[-10, 10] for i in range(3)])
ranges_dict["first_order_exp_ranges"] = np.array([[0, 3] for i in range(3)])
        
ranges_dict["second_order_coeff_ranges"] = np.array([[-2, 2] for i in range(3)])
ranges_dict["second_order_exp_ranges"] = np.array([[0, 3] for i in range(6)])

config = {"generations":200, 
          "population": 1000,
          "var_names":var_names,
          "order":"second",
          "learn_offset": False,
          "mutation_rate": [0.4, 0.5],
          "ranges_dict":ranges_dict,
          "t_subgroup_size": 3,
          "t_win_probability": 0.8,
          "percent_keep_per_gen": 0.4}

ga_poly = geneticAlgorithmPolynomial(config, datapoints[:20])

In [ ]:
ga_poly.run_algorithm()

Initializing random initial population...
Starting genetic algorithm...
Generation --    Fitness
1 --    0.021972692421180096
11 --    0.013603187700814416
21 --    0.0028851323438021936
31 --    0.0031458763147357155
41 --    2.9303222475938422e-05
51 --    4.6111004437165946e-05
61 --    4.6111004437165946e-05
71 --    4.6111004437165946e-05
81 --    4.6111004437165946e-05
91 --    4.6111004437165946e-05
101 --    4.6111004437165946e-05
111 --    4.6111004437165946e-05
121 --    4.6111004437165946e-05
131 --    2.492071170768939e-05
141 --    4.6111004437165946e-05
151 --    4.6111004437165946e-05
161 --    4.6111004437165946e-05


In [ ]:
best = ga_poly.kids[1]
best.represent()
candPoly.represent()

In [ ]:
fig, ax = plt.subplots(1)

for k in range(0,30):
    true = datapoints[k].y_var
    modeled = datapoints[k].x_var**(best.compute(datapoints[k].params_dict))
    ax.plot(datapoints[k].x_var, np.abs((modeled - true)/true), label = f"model {k}")
    ax.legend()
#ax.plot(datapoints[0].x_var, true, label = "true")
ax.set_ylim(0,2)